In [ ]:
# -*- coding: utf-8 -*-

OLMoE_random_routing_WorkFlow.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1LsxZ2chq-bEISwiGQndJGKYGJ7C_nRPI

## 1. Environment Setup

In [ ]:
import sys
import os

# Detect environment
IN_COLAB = 'google.colab' in sys.modules

print(f"Running in Google Colab: {IN_COLAB}")
print(f"Python version: {sys.version}")

# Set working directory
if IN_COLAB:
    from google.colab import drive
    print("\n📁 Mounting Google Drive...")
    drive.mount('/content/drive')

    WORK_DIR = '/content/drive/MyDrive/olmoe_random_experiments'
    REPO_DIR = '/content/drive/MyDrive/MOE-with-feature-selection'
else:
    WORK_DIR = './olmoe_random_experiments'
    REPO_DIR = None

os.makedirs(WORK_DIR, exist_ok=True)
os.chdir(WORK_DIR)
print(f"\n✅ Working directory: {os.getcwd()}")

if IN_COLAB:
    print(f"✅ Repository location: {REPO_DIR}")
from pathlib import Path

# Output directory for results
OUTPUT_DIR = Path(WORK_DIR) / 'results'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print(f"✅ Output directory: {OUTPUT_DIR}")

## 2. GPU Configuration

In [ ]:
import torch

print("=" * 70)
print("GPU CONFIGURATION")
print("=" * 70)

if torch.cuda.is_available():
    print(f"\n✅ CUDA Available")
    print(f"   CUDA Version: {torch.version.cuda}")
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

    device = 'cuda'
    torch.cuda.empty_cache()
else:
    print("\n❌ GPU not available!")
    print("\n⚠️  This notebook requires a GPU.")
    if IN_COLAB:
        print("   Enable GPU: Runtime → Change runtime type → T4/A100 GPU")
    raise Exception("GPU required for this experiment")

print(f"\n✅ Device: {device}")
print("=" * 70)

## 3. Installation

In [ ]:
%%bash
# pip install -q torch transformers datasets pandas numpy matplotlib seaborn tqdm scipy
# echo "✅ All packages installed!"

import transformers
import datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as scipy_stats

print("Package Versions:")
print(f"  torch: {torch.__version__}")
print(f"  transformers: {transformers.__version__}")
print(f"  datasets: {datasets.__version__}")
print(f"  pandas: {pd.__version__}")
print(f"  numpy: {np.__version__}")
print("\n✅ All imports successful!")

## 4. Random Routing Module Setup

In [ ]:
print("=" * 70)
print("RANDOM ROUTING MODULE SETUP")
print("=" * 70)

# ==========================================================================
# CONFIGURATION
# ==========================================================================
BRANCH = "random_ben"  # Change this to switch branches (e.g., "dev", "experiment", "feature-x")

if IN_COLAB:
    # Check if repo exists in Drive
    if os.path.exists(REPO_DIR):
        print(f"\n📂 Repository exists in Google Drive")
        print(f"   Location: {REPO_DIR}")
        print(f"   Branch: {BRANCH}")
        print(f"\n   Fetching and checking out branch '{BRANCH}'...")
        !cd {REPO_DIR} && git fetch origin && git checkout {BRANCH} && git pull origin {BRANCH}
    else:
        print(f"\n📥 Cloning repository to Google Drive (branch: {BRANCH})...")
        !git clone --branch {BRANCH} https://github.com/AliABULIEL/MOE-with-feature-selection.git {REPO_DIR}

    framework_dir = REPO_DIR
else:
    framework_dir = os.path.abspath('..')

# Add to Python path
if framework_dir not in sys.path:
    sys.path.insert(0, framework_dir)
    print(f"\n✅ Added to path: {framework_dir}")

# Verify random_routing.py exists
random_routing_file = os.path.join(framework_dir, 'random_routing.py')
if os.path.exists(random_routing_file):
    file_size = os.path.getsize(random_routing_file)
    print(f"✅ Found: random_routing.py ({file_size:,} bytes)")
else:
    raise Exception("random_routing.py not found!")

print("\n" + "=" * 70)
print("✅ MODULE READY")
print("=" * 70)

# Import Random routing functions
if 'random_routing' in sys.modules:
    del sys.modules['random_routing']

from random_routing import random_routing

print("✅ Random routing module imported successfully!")
print("\n" + "=" * 70)
print("RANDOM ROUTING API")
print("=" * 70)
print("""
Key Parameters:
- experts_amount: Number of experts to randomly select.
- sum_of_weights: Target sum for normalized weights.
                  If None, uses sum of original top-8 weights.

Usage:
  random_routing(logits, experts_amount=4, sum_of_weights=0.5)
""")

## 4.5 Import Comprehensive Framework Modules

Import the full evaluation framework with metrics, datasets, and visualizations.

In [ ]:
print("=" * 70)
print("IMPORTING COMPREHENSIVE FRAMEWORK MODULES")
print("=" * 70)

import importlib

# ✅ Import Random metrics
try:
    if 'random_routing_metrics' in sys.modules:
        importlib.reload(sys.modules['random_routing_metrics'])
    from random_routing_metrics import RandomMetricsComputer, UnifiedEvaluationMetrics, save_metrics
    print("✅ Imported RandomMetricsComputer")
    print("✅ Imported UnifiedEvaluationMetrics, save_metrics")
    metrics_computer = RandomMetricsComputer()
except ImportError as e:
    print(f"⚠️ Could not import metrics: {e}")
    metrics_computer = None

# ✅ Import Random logger
try:
    if 'random_routing_logging' in sys.modules:
        importlib.reload(sys.modules['random_routing_logging'])
    from random_routing_logging import RandomRoutingLogger
    print("✅ Imported RandomRoutingLogger")
except ImportError as e:
    print(f"⚠️ Could not import RandomRoutingLogger: {e}")
    RandomRoutingLogger = None

# ✅ Import dataset evaluation functions
try:
    if 'random_routing_evaluation' in sys.modules:
        importlib.reload(sys.modules['random_routing_evaluation'])
    from random_routing_evaluation import (
        load_wikitext, load_lambada, load_hellaswag,
        evaluate_perplexity, evaluate_lambada, evaluate_hellaswag
    )
    print("✅ Imported dataset evaluation functions")
except ImportError as e:
    print(f"⚠️ Could not import evaluation functions: {e}")

# ✅ Import visualization functions
try:
    if 'random_routing_visualization' in sys.modules:
        importlib.reload(sys.modules['random_routing_visualization'])
    from random_routing_visualization import create_comprehensive_visualization
    print("✅ Imported visualization functions")
except ImportError as e:
    print(f"⚠️ Could not import visualization functions: {e}")

print("\n" + "=" * 70)
print("✅ FRAMEWORK MODULES READY")
print("=" * 70)

## 4.6 DEBUG_MODE Configuration

Configure fast testing vs full evaluation mode.

In [ ]:
print("=" * 70)
print("DEBUG MODE CONFIGURATION")
print("=" * 70)# Toggle for fast testing vs full evaluation
DEBUG_MODE = True  # Set to True for quick testing
if DEBUG_MODE:
  # Fast testing configuration
  MAX_SAMPLES = 10
  # Very small sample for speed
  LOG_EVERY_N = 5   # Log every 5 tokens
  SAVE_PLOTS = True
  print("\n⚡ DEBUG MODE: ENABLED")
  print("   • Max samples: 10 (fast testing)")
  print("   • Logging: Every 5 tokens")
  print("   • Plots: Generated for all experiments")
else:
  # Full evaluation configuration
  MAX_SAMPLES = 200  # Full benchmark evaluation
  LOG_EVERY_N = 100
  # Log every 100 tokens for efficiency
  SAVE_PLOTS = False  # Only save summaries, not per-token logs
  print("\n🎯 PRODUCTION MODE: ENABLED")
  print("   • Max samples: 200 (full evaluation)")
  print("   • Logging: Every 100 tokens")
  print("   • Plots: Summary only")
  print("\n" + "=" * 70)
  from pathlib import Path
  if IN_COLAB:
     OUTPUT_DIR = Path(WORK_DIR) / 'random_comprehensive_results'
  else:
    OUTPUT_DIR = Path('./random_comprehensive_results')
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    (OUTPUT_DIR / 'logs').mkdir(exist_ok=True)
    (OUTPUT_DIR / 'plots').mkdir(exist_ok=True)
    (OUTPUT_DIR / 'visualizations').mkdir(exist_ok=True)
print(f"\n📁 Output directory: {OUTPUT_DIR}")

# Routing method identifier
ROUTING_METHOD = 'random'  # Used for logging and visualization

## 5. Load OLMoE Model

In [ ]:
from transformers import OlmoeForCausalLM, AutoTokenizer
from tqdm import tqdm
import time

print("=" * 70)
print("LOADING OLMoE MODEL")
print("=" * 70)

MODEL_NAME = "allenai/OLMoE-1B-7B-0924"

print(f"\nModel: {MODEL_NAME}")
print("Loading...")

start_time = time.time()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print("✅ Tokenizer loaded")

# Load model
model = OlmoeForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
model.eval()

load_time = time.time() - start_time

print(f"✅ Model loaded in {load_time:.1f}s")
print(f"\nModel Configuration:")
print(f"  • Architecture: {model.config.model_type}")
print(f"  • Hidden size: {model.config.hidden_size}")
print(f"  • Num layers: {model.config.num_hidden_layers}")
print(f"  • Num experts: {model.config.num_experts}")
print(f"  • Experts per token (Top-K): {model.config.num_experts_per_tok}")
print(f"  • Vocab size: {model.config.vocab_size}")

NUM_LAYERS = model.config.num_hidden_layers
NUM_EXPERTS = model.config.num_experts
DEFAULT_K = model.config.num_experts_per_tok

print(f"\n📊 OLMoE Routing Info:")
print(f"  • Total experts per layer: {NUM_EXPERTS}")
print(f"  • Default Top-K: {DEFAULT_K}")
print(f"  • Routing happens in {NUM_LAYERS} layers")

print("\n" + "=" * 70)
print("✅ MODEL READY")
print("=" * 70)

## 6. OLMoE Router Integration

In [ ]:
from olmoe_random_integration import RandomRoutingIntegration

# Create patcher instance
patcher = RandomRoutingIntegration(model, device=device)

print("✅ MoE block patcher initialized for Random routing")
print(f"   Ready to patch {len(patcher.moe_blocks)} OlmoeSparseMoeBlock modules")

## 6.5 VERIFICATION: Random Routing Actually Changed

**CRITICAL TEST:** Prove that Random routing is actually working.

In [ ]:
print("=" * 70)
print("VERIFICATION TEST: RANDOM ROUTING IS ACTUALLY WORKING")
print("=" * 70)

verification_prompt = "The capital of France is"
inputs = tokenizer(verification_prompt, return_tensors='pt').to(device)

print(f"\nTest prompt: '{verification_prompt}'")
print(f"\nRunning 2 tests:\n")

# TEST 1: Baseline (no patching) - should always use 8 experts
print("TEST 1: Baseline (Native Top-K=8)")
print("-" * 70)
patcher.unpatch_model()  # Ensure no patching
patcher.clear_stats()
print("  ✅ Model is not patched. Using native Top-K=8.")

# TEST 2: Random routing with experts_amount=4
print("TEST 2: Random Routing (experts_amount=4)")
print("-" * 70)
patcher.patch_model(experts_amount=4)

with torch.no_grad():
    outputs_random = model.generate(**inputs, max_new_tokens=10, do_sample=False)

stats_random = patcher.get_stats_summary()
generated_random = tokenizer.decode(outputs_random[0], skip_special_tokens=True)

print(f"  Generated: '{generated_random}'")

avg_experts = np.mean([stats_random[key]['mean'] for key in stats_random if 'counts' in key])

print(f"  Avg experts from stats: {avg_experts:.2f}")

if avg_experts == 4.0:
    print("  ✅ SUCCESS: Expert count is fixed at 4, as expected.")
    test2_pass = True
else:
    print(f"  ❌ FAILURE: Expert count is {avg_experts:.2f}, expected 4.")
    test2_pass = False

patcher.unpatch_model()

# FINAL VERDICT
print("\n" + "=" * 70)
print("VERIFICATION SUMMARY")
print("=" * 70)

if test2_pass:
    print("\n🎉 ALL CRITICAL TESTS PASSED!")
    print("\n✅ Expert counts are fixed by `experts_amount`.")
    print("✅ Output quality maintained (text is coherent).")
    print("\n🎯 RANDOM ROUTING IS ACTUALLY WORKING!")
    print("   The model NOW uses random expert selection instead of fixed Top-K!")
else:
    print("\n❌ VERIFICATION FAILED")
    print("   Expert counts are not varying as expected.")
    print("   The patching may not be working correctly.")

print("\n" + "=" * 70)

## 7. Test Prompts Configuration

In [ ]:
# Define test prompts by complexity level
TEST_PROMPTS = {
    'simple': [
        "The cat sat on the",
        "Hello, my name is",
        "The capital of France is"
    ],
    'medium': [
        "In machine learning, a neural network",
        "The process of photosynthesis involves",
        "Climate change refers to long-term shifts in"
    ],
    'complex': [
        "Explain the relationship between quantum entanglement and",
        "Compare and contrast the economic policies of",
        "The philosophical implications of consciousness suggest that"
    ],
    'technical': [
        "In Python, a decorator is a function that",
        "The time complexity of quicksort is",
        "Transformer attention mechanism computes"
    ]
}

# Flatten all prompts
ALL_PROMPTS = []
PROMPT_COMPLEXITY = []

for complexity, prompts in TEST_PROMPTS.items():
    ALL_PROMPTS.extend(prompts)
    PROMPT_COMPLEXITY.extend([complexity] * len(prompts))

print(f"Total test prompts: {len(ALL_PROMPTS)}")
print(f"\nBreakdown:")
for complexity, prompts in TEST_PROMPTS.items():
    print(f"  • {complexity.capitalize()}: {len(prompts)} prompts")

print(f"\nExample prompts:")
for complexity, prompts in list(TEST_PROMPTS.items())[:2]:
    print(f"\n  {complexity.upper()}:")
    print(f"    '{prompts[0]}'")

## 7.5 Load Benchmark Datasets

Load WikiText-2, LAMBADA, and HellaSwag for comprehensive evaluation.

In [ ]:
print("=" * 70)
print("LOADING BENCHMARK DATASETS")
print("=" * 70)

# Configure sample count
MAX_SAMPLES = 100  # Samples per dataset for comprehensive evaluation
if 'MAX_SAMPLES' not in globals():
    MAX_SAMPLES = 100  # Default if DEBUG_MODE section was not run

print(f"📊 Using MAX_SAMPLES = {MAX_SAMPLES}")
EVAL_DATASETS = {}

# Load WikiText-2
try:
    print("\n📚 Loading WikiText-2...")
    wikitext_data = load_wikitext(max_samples=MAX_SAMPLES)
    EVAL_DATASETS['wikitext'] = wikitext_data
    print(f"   ✅ Loaded {len(wikitext_data)} samples")
except Exception as e:
    print(f"   ⚠️ Failed to load WikiText: {e}")

# Load LAMBADA
try:
    print("\n📚 Loading LAMBADA...")
    lambada_data = load_lambada(max_samples=MAX_SAMPLES)
    EVAL_DATASETS['lambada'] = lambada_data
    print(f"   ✅ Loaded {len(lambada_data)} samples")
except Exception as e:
    print(f"   ⚠️ Failed to load LAMBADA: {e}")

# Load HellaSwag
try:
    print("\n📚 Loading HellaSwag...")
    hellaswag_data = load_hellaswag(max_samples=MAX_SAMPLES)
    EVAL_DATASETS['hellaswag'] = hellaswag_data
    print(f"   ✅ Loaded {len(hellaswag_data)} samples")
except Exception as e:
    print(f"   ⚠️ Failed to load HellaSwag: {e}")

print("\n" + "=" * 70)
print("✅ BENCHMARK DATASETS READY")
print("=" * 70)
print(f"\nDataset Summary:")
for name, data in EVAL_DATASETS.items():
    count = len(data) if hasattr(data, '__len__') else 0
    print(f"  • {name}: {count} samples")

## 8. Experiment Configurations

In [ ]:
from dataclasses import dataclass, field
from typing import Optional, List

@dataclass
class RoutingConfig:
    """Configuration for a routing experiment."""
    name: str
    routing_type: str  # 'baseline' or 'random'
    experts_amount: int
    sum_of_weights: Optional[float] = None
    temperature: float = 1.0
    k: Optional[int] = None # For baseline top-k
    layers_to_patch: Optional[List[int]] = None

# =========================================================================
# RANDOM ROUTING EXPERIMENT CONFIGURATIONS
# =========================================================================
configs = []

# BASELINE CONFIGURATIONS
baseline_k_values =  [4, 8, 16]

for k in baseline_k_values:
    configs.append(RoutingConfig(
        name=f'{k}experts_topk_baseline',
        routing_type='baseline',
        k=k,
        experts_amount=k
    ))

# RANDOM ROUTING CONFIGURATIONS
experts_amounts = [4, 8, 16]
sum_of_weights_options = [None, 0.5, "original"]
layers_to_patch_options = [None, [0, 15], [4, 8, 12]]

for exp_amount in experts_amounts:
    for sw in sum_of_weights_options:
        for layers in layers_to_patch_options:
            if sw == "original":
                sw_name = "swOriginal"
            else:
                sw_name = f"sw{int(sw*100)}" if sw is not None else "swDyn"

            layers_name = f"_layers_{'_'.join(map(str, layers))}" if layers is not None else ""

            configs.append(RoutingConfig(
                name=f'{exp_amount}experts_random_{sw_name}{layers_name}',
                routing_type='random',
                experts_amount=exp_amount,
                sum_of_weights=sw,
                layers_to_patch=layers
            ))

print(f"Total configurations: {len(configs)}")
print(f"  • Baselines (TopK): {len(baseline_k_values)} configs (K={baseline_k_values})")
print(f"  • Random routing: {len([c for c in configs if c.routing_type == 'random'])} configs")

print(f"\nFirst 12 configurations:")
for i, cfg in enumerate(configs[:12]):
    if cfg.routing_type == 'baseline':
        print(f"  {i+1}. {cfg.name} (TopK={cfg.k})")
    elif cfg.routing_type == 'random':
        print(f"  {i+1}. {cfg.name} (experts={cfg.experts_amount}, sum_weights={cfg.sum_of_weights})")

if len(configs) > 12:
    print(f"  ... and {len(configs) - 12} more")

print(f"\n✅ Configuration setup complete!")
print(f"   Ready to run {len(configs)} experiments")

## 9. Run Experiments

This section runs all 21 configurations on all test prompts.

In [ ]:
import json
from datetime import datetime
from typing import Dict, Any, List # Added for type hints

def run_inference(
    prompt: str,
    max_new_tokens: int = 20,
    collect_routing: bool = True
) -> Dict[str, Any]:
    """
    Run inference on a single prompt.

    Returns:
        Dictionary with:
            - generated_text: str
            - num_tokens: int
            - inference_time: float
            - routing_stats: dict (if collect_routing=True)
    """
    inputs = tokenizer(prompt, return_tensors='pt').to(device)

    start_time = time.time()

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    inference_time = time.time() - start_time

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    num_tokens = outputs.shape[1]

    result = {
        'generated_text': generated_text,
        'num_tokens': num_tokens,
        'inference_time': inference_time
    }

    if collect_routing:
        # Support different patcher APIs: prefer get_stats() if present,
        # otherwise fall back to a `stats` attribute (used elsewhere in the code).
        if hasattr(patcher, 'get_stats'):
            routing_stats = patcher.get_stats_summary()
        else:
            routing_stats = getattr(patcher, 'stats', None)
        result['routing_stats'] = routing_stats

    return result


def run_configuration(
    config: RoutingConfig,
    prompts: List[str],
    prompt_complexities: List[str],
    max_new_tokens: int = 20
) -> Dict[str, Any]:
    """
    Run a single configuration on all prompts.

    Returns:
        Dictionary with aggregated results.
    """
    # Setup routing
    if config.routing_type == 'baseline':
        # Use OLMoE's native routing (no patching)
        patcher.unpatch_model()
        print(f"  Running with OLMoE native Top-{config.k} routing")
    elif config.routing_type == 'random':
        patcher.patch_model(
            experts_amount=config.experts_amount,
            sum_of_weights=config.sum_of_weights,
            layers_to_patch=config.layers_to_patch
        )
        print(f"  Running with Random routing (experts_amount={config.experts_amount}, sum_of_weights={config.sum_of_weights}, layers_to_patch={config.layers_to_patch})")

    # Run inference on all prompts
    all_results = []
    expert_counts_all = []

    for prompt, complexity in tqdm(
        zip(prompts, prompt_complexities),
        total=len(prompts),
        desc=f"  {config.name}",
        leave=False
    ):
        patcher.stats.clear()  # Clear stats for this prompt

        result = run_inference(prompt, max_new_tokens=max_new_tokens)
        result['prompt'] = prompt
        result['complexity'] = complexity

        all_results.append(result)

        if config.routing_type == 'random' and 'routing_stats' in result:
            stats = result['routing_stats']
            if 'avg_experts' in stats:
                expert_counts_all.extend(
                    [stats['avg_experts']] * result['num_tokens']
                )

    # Aggregate results
    total_tokens = sum(r['num_tokens'] for r in all_results)
    total_time = sum(r['inference_time'] for r in all_results)

    aggregated = {
        'config_name': config.name,
        'routing_type': config.routing_type,
        'num_prompts': len(prompts),
        'total_tokens': total_tokens,
        'total_time': total_time,
        'avg_time_per_prompt': total_time / len(prompts),
        'tokens_per_second': total_tokens / total_time if total_time > 0 else 0,
        'detailed_results': all_results
    }

    # Add Random-specific metrics
    if config.routing_type == 'random':
        # Aggregate routing stats across all prompts
        all_expert_counts = []
        for r in all_results:
            if 'routing_stats' in r and r['routing_stats']:
                if 'distribution' in r['routing_stats']:
                    dist = r['routing_stats']['distribution']
                    for k, count in enumerate(dist):
                        all_expert_counts.extend([k] * count)

        if all_expert_counts:
            all_expert_counts = np.array(all_expert_counts)
            aggregated['avg_experts'] = float(np.mean(all_expert_counts))
            aggregated['std_experts'] = float(np.std(all_expert_counts))
            aggregated['min_experts'] = int(np.min(all_expert_counts))
            aggregated['max_experts'] = int(np.max(all_expert_counts))
            aggregated['median_experts'] = float(np.median(all_expert_counts))

            # Reduction vs baseline
            baseline_experts = 8  # OLMoE default
            aggregated['reduction_vs_baseline'] = float(
                (baseline_experts - aggregated['avg_experts']) / baseline_experts * 100
            )

    else:
        aggregated['k'] = config.k
        aggregated['avg_experts'] = config.k
        aggregated['std_experts'] = 0.0
        aggregated['min_experts'] = config.k
        aggregated['max_experts'] = config.k

    return aggregated

print("✅ Inference functions defined")

## 9.5 Comprehensive Benchmark Evaluation

Run evaluation on WikiText (perplexity), LAMBADA (accuracy), and HellaSwag (accuracy).

In [ ]:
# Define topk_routing function (standard baseline) - FIXED VERSION
def topk_routing(router_logits, k=8, temperature=1.0):
    """
    Standard Top-K routing (baseline).

    Args:
        router_logits: [num_tokens, num_experts]
        k: Number of experts to select
        temperature: Softmax temperature

    Returns:
        routing_weights: [num_tokens, num_experts]
        selected_experts: [num_tokens, k]
        expert_counts: [num_tokens] (always k)
    """
    import torch
    import torch.nn.functional as F

    num_tokens, num_experts = router_logits.shape
    device = router_logits.device

    # Apply temperature and softmax
    scaled_logits = router_logits / temperature
    weights = F.softmax(scaled_logits, dim=-1)

    # Select top-k experts
    topk_weights, topk_indices = torch.topk(weights, k, dim=-1)

    # Create full routing weights (zeros except for top-k)
    routing_weights = torch.zeros_like(weights)
    routing_weights.scatter_(1, topk_indices, topk_weights)

    # FIXED: No renormalization! OLMoE expects sparse weights (sum < 1.0)
    # when norm_topk_prob=False (the default).
    # The old buggy line was:
    # routing_weights = routing_weights / routing_weights.sum(dim=-1, keepdim=True)

    # Expert counts (always k for top-k)
    expert_counts = torch.full((num_tokens,), k, device=device, dtype=torch.long)

    return routing_weights, topk_indices, expert_counts

print("✅ topk_routing function defined (FIXED - no renormalization)")

SECTION 9.5: COMPREHENSIVE BENCHMARK EVALUATION WITH LOGGING
=============================================================
Evaluates ALL configurations on ALL datasets (WikiText, LAMBADA, HellaSwag)

In [ ]:
print("=" * 70)
print("COMPREHENSIVE BENCHMARK EVALUATION WITH LOGGING")
print("=" * 70)

import json # Ensure json is imported

if 'EVAL_DATASETS' not in globals() or not EVAL_DATASETS:
    print("⚠️ No datasets loaded. Skipping benchmark evaluation.")
    print("   Run Section 7.5 to load datasets first.")
    comprehensive_results = []
else:
    print(f"\nExperiment Scope:")
    print(f"  • Configurations: {len(configs)}")
    print(f"  • Datasets: {list(EVAL_DATASETS.keys())}")
    print(f"  • Samples per dataset: {MAX_SAMPLES}")
    print(f"  • Total experiments: {len(configs) * len(EVAL_DATASETS)}")

    # Configure logging based on DEBUG_MODE
    if 'LOG_EVERY_N' not in globals():
        LOG_EVERY_N = 100  # Default

    comprehensive_results = []
    benchmark_start = time.time()
    
    # Ensure logs directory exists (for checking)
    logs_dir = OUTPUT_DIR / 'logs'

    for dataset_name, dataset_data in EVAL_DATASETS.items():
        print(f"\n{'='*70}")
        print(f"EVALUATING ON: {dataset_name.upper()}")
        print(f"{'='*70}")

        for i, config in enumerate(configs):
            print(f"\n[{i+1}/{len(configs)}] {config.name} on {dataset_name}")
            print("-" * 50)

            # =============================================================
            # CHECK FOR EXISTING RESULTS (CACHE)
            # =============================================================
            experiment_name = f"{config.name}_{dataset_name}"
            log_file_path = logs_dir / f"{experiment_name}.json"
            
            if log_file_path.exists():
                try:
                    print(f"  🔍 Found existing log: {log_file_path.name}")
                    with open(log_file_path, 'r') as f:
                        cached_result = json.load(f)
                    
                    # Append cached result and skip execution
                    comprehensive_results.append(cached_result)
                    print(f"  ✅ Loaded results from cache. Skipping re-evaluation.")
                    continue 
                except Exception as e:
                    print(f"  ⚠️ Error loading cache ({e}). Re-running experiment...")

            # =============================================================
            # RUN NEW EVALUATION
            # =============================================================
            config_start = time.time()

            # Setup routing
            patcher.unpatch_model() # Fixed: Changed from unpatch() to unpatch_model()
            patcher.stats.clear()

            # Initialize logger for Random configurations only
            logger = None
            enable_logging = (config.routing_type == 'random')
            # experiment_name is already defined above
            
            if enable_logging:
                print(f"  📊 Logging enabled: {experiment_name}")

            if config.routing_type == 'baseline':
                # For baseline, we ensure the model uses its native routing.
                # RandomRoutingIntegration's 'unpatch_model' restores the original model gates,
                # which means the model's default K (typically 8 for OLMoE) will be used.
                # The patcher does NOT provide functionality to set a custom Top-K (e.g., K=4 or K=16).
                # Therefore, all 'baseline' configs will effectively run with the model's default K=8.
                patcher.unpatch_model()
                print(f"  Using OLMoE native Top-8 routing (requested K={config.k} not supported by this patcher for baseline)")
                # Temporarily override config.k to reflect actual K for results
                # config.k = 8

            elif config.routing_type == 'random':
                patcher.patch_model(
                    experts_amount=config.experts_amount,
                    sum_of_weights=config.sum_of_weights,
                    layers_to_patch=config.layers_to_patch
                )
                print(f"  Using Random routing (experts_amount={config.experts_amount}, sum_of_weights={config.sum_of_weights}, layers_to_patch={config.layers_to_patch})")

            # Initialize result with common fields
            result = {
                'config_name': config.name,
                'dataset': dataset_name,
            }

            if config.routing_type == 'baseline':
                result['routing_type'] = 'topk'
                result['k_or_max_k'] = config.k
            elif config.routing_type == 'random':
                result['routing_type'] = 'random'
                result['k_or_max_k'] = config.experts_amount


            # ⭐ CRITICAL: Call start_sample() BEFORE evaluation for proper logging
            patcher.start_sample()

            try:
                # =============================================================
                # EVALUATE BASED ON DATASET TYPE
                # =============================================================

                if dataset_name == 'wikitext':
                    eval_result = evaluate_perplexity(
                        model=model,
                        tokenizer=tokenizer,
                        dataset=dataset_data,
                        patcher=patcher,  # ✅ NEW: Pass patcher for logging
                        device=device,
                        max_length=512,
                        # ✅ NEW: Logging parameters
                        log_routing=enable_logging,
                        output_dir=str(OUTPUT_DIR),
                        experiment_name=experiment_name,
                        log_every_n=LOG_EVERY_N
                    )
                    # Use eval_result['perplexity'] as the primary perplexity value
                    result['perplexity'] = eval_result['perplexity']
                    # No longer attempting to access missing keys like perplexity_token_weighted
                    print(f"  ✅ Perplexity (token-weighted): {eval_result['perplexity']:.2f}")

                elif dataset_name == 'lambada':
                    eval_result = evaluate_lambada(
                        model=model,
                        tokenizer=tokenizer,
                        dataset=dataset_data,
                        patcher=patcher,  # ✅ NEW: Pass patcher
                        device=device,
                        # ✅ NEW: Logging parameters
                        log_routing=enable_logging,
                        output_dir=str(OUTPUT_DIR),
                        experiment_name=experiment_name,
                        log_every_n=LOG_EVERY_N
                    )
                    result['lambada_accuracy'] = eval_result['accuracy']
                    print(f"  ✅ LAMBADA Accuracy: {eval_result['accuracy']:.4f}")

                # elif dataset_name == 'hellaswag':
                #     eval_result = evaluate_hellaswag(
                #         model=model,
                #         tokenizer=tokenizer,
                #         dataset=dataset_data,
                #         patcher=patcher,  # ✅ NEW: Pass patcher
                #         device=device,
                #         # ✅ NEW: Logging parameters
                #         log_routing=enable_logging,
                #         output_dir=str(OUTPUT_DIR),
                #         experiment_name=experiment_name,
                #         log_every_n=LOG_EVERY_N
                #     )
                #     # ✅ NEW: Capture BOTH raw and normalized accuracy
                #     result['hellaswag_accuracy'] = eval_result['accuracy']  # Raw (backward compat)
                #     result['hellaswag_accuracy_raw'] = eval_result['accuracy_raw']
                #     result['hellaswag_accuracy_normalized'] = eval_result['accuracy_normalized']
                #     print(f"  ✅ HellaSwag Accuracy (raw): {eval_result['accuracy_raw']:.4f}")
                #     print(f"  ✅ HellaSwag Accuracy (normalized): {eval_result['accuracy_normalized']:.4f}")

            except Exception as e:
                print(f"  ❌ Evaluation failed: {e}")
                import traceback
                print(traceback.format_exc())
                result['error'] = str(e)

            # =============================================================
            # GET ROUTING STATISTICS
            # =============================================================

            # Changed from get_stats() to get_stats_summary()
            stats = patcher.get_stats_summary()

            # Determine k_val for statistics based on config type
            if config.routing_type == 'baseline':
                k_val = config.k # this is the overridden value (8)
            elif config.routing_type == 'random':
                k_val = config.experts_amount # for random, experts_amount is the effective K
            else:
                k_val = 8 # Fallback to default OLMoE K

            if stats:
                result['avg_experts'] = stats.get('avg_experts', k_val)
                result['std_experts'] = stats.get('std_experts', 0)
                result['min_experts'] = stats.get('min_experts', k_val)
                result['max_experts'] = stats.get('max_experts', k_val)

                # Compute additional metrics if metrics_computer exists
                expert_counts = np.array(patcher.stats.get('expert_counts', []))
                if len(expert_counts) > 0:
                    # Compute metrics inline (no external metrics_computer needed)

                    # Adaptive range: max - min experts
                    result['adaptive_range'] = int(np.max(expert_counts) - np.min(expert_counts))

                    # Ceiling hit rate: % of tokens hitting max_k
                    result['ceiling_hit_rate'] = float((expert_counts >= k_val).sum() / len(expert_counts) * 100)

                    # Floor hit rate: % of tokens at min_k
                    # For random routing, min_k is effectively k_val unless explicitly varied.
                    result['floor_hit_rate'] = float((expert_counts <= k_val).sum() / len(expert_counts) * 100)

                    # Mid range rate
                    result['mid_range_rate'] = 100.0 - result['ceiling_hit_rate'] - result['floor_hit_rate']

                    # Selection entropy
                    try:
                        from scipy.stats import entropy as scipy_entropy
                        counts_dist = np.bincount(expert_counts.astype(int), minlength=65)
                        counts_dist = counts_dist / (counts_dist.sum() + 1e-10)
                        result['selection_entropy'] = float(scipy_entropy(counts_dist + 1e-10))
                        max_entropy = np.log(k_val) if k_val > 1 else 1.0
                        result['normalized_entropy'] = result['selection_entropy'] / max_entropy
                    except:
                        result['selection_entropy'] = 0.0
                        result['normalized_entropy'] = 0.0

                    # Expert activation ratio: avg_experts / total_experts (64)
                    result['expert_activation_ratio'] = result['avg_experts'] / 64.0

                    # FLOPs reduction vs baseline (k=8)
                    result['flops_reduction_pct'] = (8.0 - result['avg_experts']) / 8.0 * 100

                    # Reduction vs baseline (positive = fewer experts = more efficient)
                    result['reduction_vs_baseline'] = (8 - result['avg_experts']) / 8 * 100

                    print(f"  📊 Avg Experts: {result.get('avg_experts', 'N/A'):.2f} ± {result.get('std_experts', 0):.2f}")
                    print(f"  📊 Range: [{result.get('min_experts', 'N/A')}, {result.get('max_experts', 'N/A')}]")
                    print(f"  📊 Reduction vs Baseline: {result.get('reduction_vs_baseline', 0):.1f}%")
            else:
                # For configurations where stats were not collected (e.g., initial baseline run)
                result['avg_experts'] = k_val
                result['std_experts'] = 0
                result['min_experts'] = k_val
                result['max_experts'] = k_val
                result['adaptive_range'] = 0
                result['ceiling_hit_rate'] = 100.0
                result['floor_hit_rate'] = 0.0
                result['mid_range_rate'] = 0.0
                result['reduction_vs_baseline'] = (8 - k_val) / 8 * 100

            # =============================================================
            # SAVE LOGS AND GENERATE PLOTS
            # =============================================================

            # if logger is not None:
            #     try:
            #         logger.save_logs()

            #         if 'SAVE_PLOTS' in globals() and SAVE_PLOTS:
            #             logger.generate_plots()
            #             print(f"  📊 Generated plots")

            #         summary = logger.get_summary()
            #         if summary:
            #             result['logger_summary'] = summary

            #         logger.clear()
            #     except Exception as e:
            #         print(f"  ⚠️ Logging/plotting failed: {e}")

            config_time = time.time() - config_start
            result['elapsed_time'] = config_time
            print(f"  ⏱️ Completed in {config_time:.1f}s")

            comprehensive_results.append(result)
            
            # --- Auto-save individual result immediately (Double safety) ---
            try:
                # Save immediately to avoid losing progress if kernel crashes
                summary_file = logs_dir / f"{experiment_name}.json"
                with open(summary_file, 'w') as f:
                    json.dump(result, f, indent=2, default=str)
            except Exception as e:
                print(f"  ⚠️ Failed to save intermediate log: {e}")

            # Clear GPU cache
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

    # Ensure unpatched
    patcher.unpatch_model() # Fixed: Changed from unpatch() to unpatch_model()

    benchmark_time = time.time() - benchmark_start

    # =========================================================================
    # SUMMARY
    # =========================================================================

    print("\n" + "=" * 70)
    print("COMPREHENSIVE BENCHMARK EVALUATION COMPLETE!")
    print("=" * 70)
    print(f"\n⏱️ Total time: {benchmark_time / 60:.1f} minutes")
    print(f"📊 Experiments completed: {len(comprehensive_results)}")

    # Create summary DataFrame
    results_df = pd.DataFrame(comprehensive_results)

    # =========================================================================
    # DISPLAY RESULTS BY DATASET
    # =========================================================================

    print("\n" + "=" * 70)
    print("RESULTS BY DATASET")
    print("=" * 70)

    for dataset_name in EVAL_DATASETS.keys():
        dataset_results = results_df[results_df['dataset'] == dataset_name]

        print(f"\n📊 {dataset_name.upper()}")
        print("-" * 50)

        if dataset_name == 'wikitext':
            cols = ['config_name', 'avg_experts', 'perplexity', 'reduction_vs_baseline']
            if 'perplexity' in dataset_results.columns:
                display_df = dataset_results[cols].copy()
                display_df['perplexity'] = display_df['perplexity'].round(2)
                display_df['avg_experts'] = display_df['avg_experts'].round(2)
                display_df['reduction_vs_baseline'] = display_df['reduction_vs_baseline'].round(1)
                print(display_df.to_string(index=False))

        elif dataset_name == 'lambada':
            cols = ['config_name', 'avg_experts', 'lambada_accuracy', 'reduction_vs_baseline']
            if 'lambada_accuracy' in dataset_results.columns:
                display_df = dataset_results[cols].copy()
                display_df['lambada_accuracy'] = display_df['lambada_accuracy'].round(4)
                display_df['avg_experts'] = display_df['avg_experts'].round(2)
                display_df['reduction_vs_baseline'] = display_df['reduction_vs_baseline'].round(1)
                print(display_df.to_string(index=False))

        # elif dataset_name == 'hellaswag':
        #     cols = ['config_name', 'avg_experts', 'hellaswag_accuracy', 'reduction_vs_baseline']
        #     if 'hellaswag_accuracy' in dataset_results.columns:
        #         display_df = dataset_results[cols].copy()
        #         display_df['hellaswag_accuracy'] = display_df['hellaswag_accuracy'].round(4)
        #         display_df['avg_experts'] = display_df['avg_experts'].round(2)
        #         display_df['reduction_vs_baseline'] = display_df['reduction_vs_baseline'].round(1)
        #         print(display_df.to_string(index=False))

    # =========================================================================
    # SAVE RESULTS
    # =========================================================================

    results_path = OUTPUT_DIR / 'comprehensive_results_all_datasets.csv'
    results_df.to_csv(results_path, index=False)
    print(f"\n✅ Results saved to: {results_path}")

    # =========================================================================
    # KEY FINDINGS SUMMARY
    # =========================================================================

    print("\n" + "=" * 70)
    print("KEY FINDINGS")
    print("=" * 70)

    # Compare baseline vs best random config
    baseline_results = results_df[results_df['routing_type'] == 'topk']
    random_results = results_df[results_df['routing_type'] == 'random']

    if len(baseline_results) > 0 and len(random_results) > 0:
        print("\n📈 Baseline vs Random Routing Comparison:")
        print("-" * 50)

        for dataset_name in EVAL_DATASETS.keys():
            baseline_ds = baseline_results[baseline_results['dataset'] == dataset_name]
            random_ds = random_results[random_results['dataset'] == dataset_name]

            if len(baseline_ds) > 0 and len(random_ds) > 0:
                baseline_row = baseline_ds.iloc[0]

                print(f"\n  {dataset_name.upper()}:")
                print(f"    Baseline (TopK=8): avg_experts=8.00")

                if dataset_name == 'wikitext' and 'perplexity' in baseline_row:
                    baseline_ppl = baseline_row['perplexity']
                    print(f"      Perplexity: {baseline_ppl:.2f}")

                    for _, random_row in random_ds.iterrows():
                        random_ppl = random_row.get('perplexity', float('inf'))
                        ppl_diff = ((random_ppl - baseline_ppl) / baseline_ppl) * 100
                        print(f"    {random_row['config_name']}: avg={random_row['avg_experts']:.2f}, PPL={random_ppl:.2f} ({ppl_diff:+.1f}%)")

                elif dataset_name == 'lambada' and 'lambada_accuracy' in baseline_row:
                    baseline_acc = baseline_row['lambada_accuracy']
                    print(f"      Accuracy: {baseline_acc:.4f}")

                    for _, random_row in random_ds.iterrows():
                        random_acc = random_row.get('lambada_accuracy', 0)
                        acc_diff = ((random_acc - baseline_acc) / baseline_acc) * 100 if baseline_acc > 0 else 0
                        print(f"    {random_row['config_name']}: avg={random_row['avg_experts']:.2f}, Acc={random_acc:.4f} ({acc_diff:+.1f}%)")

                # elif dataset_name == 'hellaswag' and 'hellaswag_accuracy' in baseline_row:
                #     baseline_acc = baseline_row['hellaswag_accuracy']
                #     print(f"      Accuracy: {baseline_acc:.4f}")

                #     for _, random_row in random_ds.iterrows():
                #         random_acc = random_row.get('hellaswag_accuracy', 0)
                #         acc_diff = ((random_acc - baseline_acc) / baseline_acc) * 100 if baseline_acc > 0 else 0
                #         print(f"    {random_row['config_name']}: avg={random_row['avg_experts']:.2f}, Acc={random_acc:.4f} ({acc_diff:+.1f}%)")


    print("\n" + "=" * 70)

## 10. Save Comprehensive Results

SECTION: SAVING COMPREHENSIVE RESULTS
=====================================
Copy this entire cell to replace the broken one in your notebook.

In [ ]:
print("=" * 70)
print("SAVING COMPREHENSIVE RESULTS")
print("=" * 70)

# Create output directories
from pathlib import Path
import json

if 'OUTPUT_DIR' not in globals():
    if IN_COLAB:
        OUTPUT_DIR = Path(WORK_DIR) / 'random_comprehensive_results'
    else:
        OUTPUT_DIR = Path('./random_comprehensive_results')

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
(OUTPUT_DIR / 'logs').mkdir(exist_ok=True)
(OUTPUT_DIR / 'visualizations').mkdir(exist_ok=True)

print(f"📁 Output directory: {OUTPUT_DIR}")

# Prefer comprehensive benchmark results, fallback to test prompt results
if 'comprehensive_results' in globals() and comprehensive_results:
    results_to_save = comprehensive_results
    print("✅ Using comprehensive benchmark results")
elif 'all_experiment_results' in globals() and all_experiment_results:
    # Add missing columns to test prompt results for compatibility
    for result in all_experiment_results:
        if 'routing_type' in result and result['routing_type'] == 'baseline':
            result['routing_type'] = 'topk'  # Match viz expectations
        if 'k' in result and 'k_or_max_k' not in result:
            result['k_or_max_k'] = result['k']
        elif 'max_k' in result and 'k_or_max_k' not in result:
            result['k_or_max_k'] = result['max_k']
        if 'dataset' not in result:
            result['dataset'] = 'test_prompts'
        if 'mid_range_rate' not in result and 'ceiling_hit_rate' in result:
            result['mid_range_rate'] = 100 - result.get('ceiling_hit_rate', 0) - result.get('floor_hit_rate', 0)
    results_to_save = all_experiment_results
    print("⚠️ Using test prompt results (no benchmark data)")
else:
    print("⚠️ No results to save!")
    results_to_save = []

if results_to_save:
    # Create comprehensive DataFrame
    results_df = pd.DataFrame(results_to_save)

    # Save CSV
    csv_path = OUTPUT_DIR / 'random_comprehensive_results.csv'
    results_df.to_csv(csv_path, index=False)
    print(f"✅ Saved CSV: {csv_path}")

    # Save JSON
    json_path = OUTPUT_DIR / 'random_comprehensive_results.json'
    results_df.to_json(json_path, orient='records', indent=2)
    print(f"✅ Saved JSON: {json_path}")

    # Save per-config summary JSONs (dual file logging)
    logs_dir = OUTPUT_DIR / 'logs'
    for result in results_to_save:
        config_name = result.get('config_name', 'unknown')
        dataset = result.get('dataset', 'mixed')

        summary_file = logs_dir / f"{config_name}_{dataset}.json"
        with open(summary_file, 'w') as f:
            json.dump(result, f, indent=2, default=str)

    print(f"✅ Saved {len(results_to_save)} individual log files to {logs_dir}")

    # Display summary tables
    print("\n" + "=" * 70)
    print("RESULTS SUMMARY")
    print("=" * 70)

    # Show top results
    if 'avg_experts' in results_df.columns:
        print("\nTop 10 configurations by average experts:")
        display_cols = ['config_name', 'avg_experts']
        for col in ['reduction_vs_baseline', 'dataset']:
            if col in results_df.columns:
                display_cols.append(col)
        top_df = results_df.nsmallest(10, 'avg_experts')[display_cols]
        print(top_df.to_string(index=False))

    # Quality metrics by dataset
    if 'dataset' in results_df.columns:
        for dataset in results_df['dataset'].unique():
            subset = results_df[results_df['dataset'] == dataset]
            print(f"\n📊 {dataset.upper()} Results:")

            display_cols = ['config_name']
            if 'perplexity' in subset.columns:
                display_cols.append('perplexity')
            if 'lambada_accuracy' in subset.columns:
                display_cols.append('lambada_accuracy')
            # if 'hellaswag_accuracy' in subset.columns:
            #     display_cols.append('hellaswag_accuracy')
            if 'avg_experts' in subset.columns:
                display_cols.append('avg_experts')

            if len(display_cols) > 1:
                print(subset[display_cols].head(10).to_string(index=False))

else:
    print("⚠️ No results to save!")
    results_df = pd.DataFrame()  # Create empty DataFrame

print("\n" + "=" * 70)

# Routing method identifier
ROUTING_METHOD = 'random'  # Used for logging and visualization

## 11. Comprehensive Visualizations

In [ ]:
print("=" * 70)
print("GENERATING COMPREHENSIVE VISUALIZATIONS")
print("=" * 70)

if 'results_df' not in globals() or results_df is None or len(results_df) == 0:
    print("\n⚠️ No results DataFrame available")
    print("   Run Sections 9.5 and 10 first.")
elif 'create_comprehensive_visualization' not in globals():
    print("\n⚠️ Visualization function not loaded")
    print("   Run Section 4.5 to import framework modules.")
else:
    # Validate required columns
    required_cols = ['routing_type', 'k_or_max_k', 'dataset', 'avg_experts']
    missing_cols = [c for c in required_cols if c not in results_df.columns]

    if missing_cols:
        print(f"\n⚠️ Missing columns for full visualization: {missing_cols}")
        print("Falling back to basic visualization...")

        # Fallback to basic visualization
        import matplotlib.pyplot as plt
        import seaborn as sns

        sns.set_style('whitegrid')
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        fig.suptitle('OLMoE Random Routing Analysis', fontsize=16, fontweight='bold')

        # Filter Random results
        random_df = results_df[results_df['routing_type'] == 'random'] if 'routing_type' in results_df.columns else results_df

        # Plot 1: Average experts
        if 'avg_experts' in results_df.columns:
            ax1 = axes[0, 0]
            top_10 = results_df.nsmallest(10, 'avg_experts')
            ax1.barh(range(len(top_10)), top_10['avg_experts'])
            ax1.set_yticks(range(len(top_10)))
            ax1.set_yticklabels(top_10['config_name'], fontsize=8)
            ax1.set_title('Top 10: Fewest Experts')
            ax1.set_xlabel('Average Experts')

        # Plot 2: Throughput
        if 'tokens_per_second' in results_df.columns:
            ax2 = axes[0, 1]
            top_10 = results_df.nlargest(10, 'tokens_per_second')
            ax2.barh(range(len(top_10)), top_10['tokens_per_second'])
            ax2.set_yticks(range(len(top_10)))
            ax2.set_yticklabels(top_10['config_name'], fontsize=8)
            ax2.set_title('Top 10: Throughput')
            ax2.set_xlabel('Tokens/Second')

        # Plot 3: Reduction vs baseline
        if 'reduction_vs_baseline' in random_df.columns and len(random_df) > 0:
            ax3 = axes[1, 0]
            top_10 = random_df.nlargest(10, 'reduction_vs_baseline')
            ax3.barh(range(len(top_10)), top_10['reduction_vs_baseline'])
            ax3.set_yticks(range(len(top_10)))
            ax3.set_yticklabels(top_10['config_name'], fontsize=8)
            ax3.set_title('Top 10: Expert Reduction')
            ax3.set_xlabel('Reduction (%)')

        # Plot 4: Summary stats
        ax4 = axes[1, 1]
        ax4.axis('off')
        summary_text = f"""
        Total Configs: {len(results_df)}
        Random Configs: {len(random_df)}
        """
        if 'avg_experts' in results_df.columns:
            summary_text += f"\nMin Avg Experts: {results_df['avg_experts'].min():.2f}"
            summary_text += f"\nMax Avg Experts: {results_df['avg_experts'].max():.2f}"
        ax4.text(0.1, 0.5, summary_text, fontsize=12, va='center')

        plt.tight_layout()
        viz_path = OUTPUT_DIR / 'visualizations' / 'basic_analysis.png'
        plt.savefig(viz_path, dpi=300, bbox_inches='tight')
        print(f"✅ Saved basic visualization: {viz_path}")
        plt.show()
    else:
        # Use comprehensive visualization
        try:
            viz_path = create_comprehensive_visualization(
                results_df=results_df,
                output_path=str(OUTPUT_DIR / 'visualizations' / 'random_comprehensive_comparison.png')
            )

            if viz_path:
                print(f"✅ Saved comprehensive visualization: {viz_path}")

                # Display the visualization
                from IPython.display import Image, display
                if Path(viz_path).exists():
                    display(Image(filename=str(viz_path)))
            else:
                print("⚠️ Visualization not created")
        except Exception as e:
            print(f"⚠️ Comprehensive visualization failed: {e}")
            import traceback
            print(traceback.format_exc())
            print("\nTry running the basic visualization fallback above.")

print("\n" + "=" * 70)
print("✅ VISUALIZATIONS COMPLETE")
print("=" * 70)

## 12. Statistical Analysis

In [ ]:
print("=" * 70)
print("COMPREHENSIVE STATISTICAL ANALYSIS")
print("=" * 70)

# Define random_df and baseline_df from results
if 'results_df' not in globals() or results_df is None or len(results_df) == 0:
    print("\n⚠️ No results DataFrame available for analysis")
    print("   Run Sections 9.5 and 10 first to generate results.")
else:
    # Filter Random and baseline results
    random_df = results_df[results_df['routing_type'] == 'random'].copy() if 'routing_type' in results_df.columns else pd.DataFrame()
    baseline_df = results_df[results_df['routing_type'] == 'topk'].copy() if 'routing_type' in results_df.columns else pd.DataFrame()

    print(f"\nResults breakdown:")
    print(f"  • Total configurations: {len(results_df)}")
    print(f"  • Baseline (TopK): {len(baseline_df)}")
    print(f"  • random routing: {len(random_df)}")

    if len(random_df) == 0:
        print("\n⚠️ No Random results found. Skipping Random-specific analysis.")
    else:
        # =====================================================================
        # 1. BASELINE COMPARISON
        # =====================================================================
        print("\n1. BASELINE COMPARISON")
        print("-" * 50)

        if len(baseline_df) > 0 and 'avg_experts' in baseline_df.columns:
            print("\nBaseline Configurations:")
            baseline_cols = ['config_name', 'avg_experts']
            if 'dataset' in baseline_df.columns:
                baseline_cols.append('dataset')
            if 'perplexity' in baseline_df.columns:
                baseline_cols.append('perplexity')
            display_cols = [c for c in baseline_cols if c in baseline_df.columns]
            print(baseline_df[display_cols].to_string(index=False))

        # =====================================================================
        # 2. RANDOM ROUTING ANALYSIS
        # =====================================================================
        print("\n\n2. RANDOM ROUTING ANALYSIS")
        print("-" * 50)

        # Best by reduction (most efficient)
        if 'reduction_vs_baseline' in random_df.columns:
            print("\nTop 5 by Expert Reduction:")
            best_reduction = random_df.nlargest(5, 'reduction_vs_baseline')
            display_cols = ['config_name', 'avg_experts', 'reduction_vs_baseline']
            if 'max_k' in best_reduction.columns:
                display_cols.append('max_k')
            display_cols = [c for c in display_cols if c in best_reduction.columns]
            print(best_reduction[display_cols].to_string(index=False))
        else:
            print("\n⚠️ Column 'reduction_vs_baseline' not found")

        # Best by low ceiling hit rate (not constrained)
        if 'ceiling_hit_rate' in random_df.columns:
            print("\n\nTop 5 by Low Ceiling Hit Rate (unconstrained):")
            best_unconstrained = random_df.nsmallest(5, 'ceiling_hit_rate')
            display_cols = ['config_name', 'avg_experts', 'ceiling_hit_rate']
            if 'max_k' in best_unconstrained.columns:
                display_cols.append('max_k')
            display_cols = [c for c in display_cols if c in best_unconstrained.columns]
            print(best_unconstrained[display_cols].to_string(index=False))
        else:
            print("\n⚠️ Column 'ceiling_hit_rate' not found")

        # Best by adaptive range (most dynamic)
        if 'adaptive_range' in random_df.columns:
            print("\n\nTop 5 by Adaptive Range (most dynamic):")
            best_adaptive = random_df.nlargest(5, 'adaptive_range')
            display_cols = ['config_name', 'adaptive_range', 'avg_experts']
            if 'dataset' in best_adaptive.columns:
                display_cols.append('dataset')
            display_cols = [c for c in display_cols if c in best_adaptive.columns]
            print(best_adaptive[display_cols].to_string(index=False))

print("\n" + "=" * 70)

## 13. Generate Report

In [ ]:
if 'results_df' not in globals() or results_df is None or len(results_df) == 0:
    print("⚠️ No results to generate report from")
    print("   Run Sections 9.5 and 10 first.")
else:
    report_path = OUTPUT_DIR / 'random_routing_comprehensive_report.md'

    with open(report_path, 'w') as f:
        from datetime import datetime
        f.write("# OLMoE Random Routing Comprehensive Evaluation Report\n\n")
        f.write(f"**Date:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        f.write(f"**Model:** {MODEL_NAME}\n\n")

        f.write("---\n\n")

        f.write("## Executive Summary\n\n")
        f.write(f"- **Configurations tested:** {len(results_df)}\n")
        if 'routing_type' in results_df.columns:
            baseline_count = len(results_df[results_df['routing_type'] == 'topk'])
            random_count = len(results_df[results_df['routing_type'] == 'random'])
            f.write(f"  - Baselines: {baseline_count}\n")
            f.write(f"  - Random variants: {random_count}\n")
        if 'dataset' in results_df.columns:
            datasets = results_df['dataset'].unique().tolist()
            f.write(f"- **Datasets evaluated:** {datasets}\n")
        f.write("\n")

        f.write("---\n\n")

        f.write("## Key Findings\n\n")

        # Best configurations
        random_df = results_df[results_df['routing_type'] == 'random'] if 'routing_type' in results_df.columns else results_df

        if len(random_df) > 0 and 'avg_experts' in random_df.columns:
            best = random_df.nsmallest(1, 'avg_experts').iloc[0]
            f.write("### Best Efficiency (Fewest Experts)\n\n")
            f.write(f"- **Configuration:** {best['config_name']}\n")
            f.write(f"- **Avg Experts:** {best['avg_experts']:.2f}\n\n")

            if len(random_df) > 0 and 'reduction_vs_baseline' in random_df.columns:

                best_red = random_df.nlargest(1, 'reduction_vs_baseline').iloc[0]

                f.write("### Best Expert Reduction\n\n")

                f.write(f"- **Configuration:** {best_red['config_name']}\n")

                f.write(f"- **Reduction:** {best_red['reduction_vs_baseline']:.1f}%\n")

                f.write(f"- **Avg Experts:** {best_red['avg_experts']:.2f}\n\n")

        

                f.write("---\n\n")

        

                f.write("## Full Results\n\n")

                f.write(results_df.to_markdown(index=False))

                f.write("\n\n")

        

                f.write("---\n\n")

                f.write(f"Generated by Random Routing Framework\n")

                f.write(f"Output directory: {OUTPUT_DIR}\n")

        

            print(f"✅ Generated comprehensive report: {report_path}")